# CNN, RNN, エンコーダー, バッチ学習およびAttention機構の手法の調べ物レポート

## 目的

前回のレポートでは村田さん、丸野さんの論文での実験内容に対するプログラムと結果の再現および手法の確認を行った。また、新たな手法として、GRUモデルを用いたモデルの実装および過学習を防ぐためのDropout層を追加したGRUモデルの動作確認を行った。結果としては丸野さんの論文同様に機械学習を用いた人間計算可能なパスワードの突破が不可能であるという結論になった。このレポートでは、前のレポートを通してCNN, RNN等の機械学習モデルの詳細およびバッチなどの学習手法について調べる必要があると考えられるため、それらについてまとめる。

## CNN

CNNとは、画像認識や画像編集、音声認識などの問題解決に活用される機械学習モデルである。このモデルは主にPooling（プーリング）層と呼ばれる層とConvolution（畳み込み）層と呼ばれる層によって構成される。これらの層の役割は後述する。このモデルの特徴としては次の二点が挙げられる。

- Convolution層での処理が画像編集などで使われるフィルタ処理に非常に類似している
- 多層パーセプトロンでは二次元データを扱う際に一次元データに変換して扱っていたのに対して、CNNでは二次元データを二次元データの形式のままで処理できる

これらの特徴は、後述する二つの層の仕組みによって成り立っている。なお、このモデルの構造は、入力層、出力層に加えて、中間層としてConvolution層とPooling層が繰り返される点にある。

### Convolution層

Convolution層は畳み込み層とも呼ばれる層である。この層は入力として二次元配列を受け取ることが可能である。二次元配列の大きさ(n, m)とし、二次元配列の要素をinput(i, j)で指定するとする。Convolution層での処理では、この二次元配列に対して、フィルタと呼ばれる二次元配列を畳み込む処理を行う。このフィルタは、Convolution層の出力output(サイズ(n-2, m-2))に対して次のような出力を与える。

output(i, j) = i_0 * input(i-1, j-1) + i_1 * input(i, j-i) + i_2 * input(i+1, j-1) + i_3 * input(i-1, j) + i_4 * input(i, j) + i_5 * input(i+1, j) + i_6 * input(i-1, j+1) + i_7 * input(i, j+1) + i_8 * input(i+1, j+1)

画像処理などにおいては、i_0. i_1などを事前に決定しておくことで、物体の境界線を検知するエッジフィルタなどを実装できることが知られている。画像処理のフィルタと違う点は、CNNにおけるiの値は学習対象であるということである。

### Pooling層

Pooling層はダウンサンプリング層とも呼ばれる。Pooling層では、Convolution層同様に二次元配列を入力として受け取り、二次元配列を出力とする。

## RNN

RNNとは、時系列データの解析などに活用される機械学習モデルである。自然言語処理や数値予測などに活用されている。最初に提唱されたSimpleRNN以外に、勾配消失問題を解決したLSTM(Long short term memory)やLSTMから計算量を減らしつつ同様の特徴を実現したGRU, 未来のデータがわかる場合に間のデータの予測に活用できるBi-Directional RNNや言語生成で注目を浴びているAttention RNNという手法がある。それぞれの手法の特徴について後述する。

### SimpleRNN

SimpleRNNはRNNの中で最も単純な実装である。RNNの基本的なネットワーク構成は多層パーセプトロン等のネットワーク同様に入力層、中間層、出力層を持つ構造であり、内部構造もそれらに近しい。RNNの特徴としては、中間層の内部に"コンテキストを持つ"層があるという点である。
入力層に入力されるデータは時刻t=iの時点でのデータであるとする。このデータ自体は特定時刻のデータのみしか持たない。ただし、コンテキストを持つ層が以前の入力であるt=i-1, i-2, ...の情報を覚えているという点がRNNの特徴である。
時点tにおける入力x(t), 出力y(t), 状態s(t)とするとこのネットワークは次のように記述できる。

y(t) = f(s(t), x(t))
s(t) = (s(t-1), x(t))

### LSTM(長短期記憶、Long short term memory)

前述したSimpleRNNでも時系列データの解析は可能であったが、勾配消失問題が発生することにより一定以上過去のデータに依存する結果を学習することができないという欠点があった。この短所を乗り越えたのがLSTMとよばれる仕組みである。LSTMの基本構造はSimpleRNNと類似しているが、次のような特徴がある。

- x(t)とs(t)の間に"入力ゲート"という機構がある
- s(t)とs(t-1)の間に"忘却ゲート"という機構がある
- s(t)とy(t)の間に"出力ゲート"という機構がある

これらゲートの内容はシグモイド関数である。学習を行った結果としてこれらゲートのパラメーターが調整され、どの内容を記憶するかが決定される。

### GRU(Gated Recurrent Unit)

GRUはLSTMの計算量の多さを解決したモデルである。このモデルは次のような特徴を持つ。

- y(t)とs(t)の間に"出力ゲート"という機構がある
- s(t)とs(t-1)の間に"更新ゲート"という機構がある

基本的な仕組みとしてはLSTMと同様である。GRUはLSTM同等の機能をゲート二つで実現しているため、計算量が多少少なくなっている。

### Bi-Directional RNN

Bi-Directional RNNの特徴は、SimpleRNNにおいてs(t)が過去の値に依存することに対して、同様の手法でs'(t+1)に依存するs'(t)の層が存在する点である。SimpleRNNでは過去の情報のみを参考に学習・推論を行っているが、未来の値もわかっている場合にはBi-Directional RNNを用いた推論を行うことが可能である。具体的な事例としては、動画のフレーム間の映像の推論を行い動画のフレームレートを上げることなどが考えられる。


## Attention機構とは

Attention機構とは、機械学習での学習や推論を行う場合に注目するべきデータのみを取り出すための機構である。Attention機構を機械学習ネットワークに組み込むことで、画像認識に映り込んだ背景などの関係ないデータに影響されにくい学習・推論を行うことが可能である。具体的な例を後述する。

### 画像認識にAttention機構を組み合わせる例

画像認識の分野では、認識を行うネットワークの前段としてAttention機構が用いられる場合がある。この場合のAttention機構はConvolution層とPooling層一層ずつで構成されており、この層から1チャネルの二値の値が画素数と同じ数出力される。入力データに対してこの出力値と積を取ることで注目する必要がある情報が抜き出される。

## アンサンブル学習とは

これまで述べてきた手法では、一つの項目を予測するために一つだけの機械学習モデルを活用していた。これに対して、複数の弱い学習器を用いて多数決的に予測を行う手法がアンサンブル学習である。この手法は、一つの学習器では予測が難しい場合に、複数の学習器を用いることで予測精度を向上させることができる。代表的な手法として、バギング、ブースティング、スタッキングがある。

### バギング

バギングと呼ばれる手法では、複数の学習器を用いた多数決を用いて予測を行う。ここで、学習を行う際の特徴として、それぞれの学習器に重複しない学習データのサブセットを学習させる特徴がある。複数の学習器を並列に学習できるため、学習が早いという特徴がある。代表的な手法として、複数の決定木を用いたバギング手法であるランダムフォレストが挙げられる。

## 今後の展望

以前の実験から、人間計算可能なパスワードについてLSTMやMLPなどの各モデルで10%-15%程度の正解率が達成できることがわかっている。これらのことから、LSTMやMLPといった複数の機械学習モデルを用いたアンサンブル学習を試すことで精度が向上することが期待される。また、今回の学習データには異常データはないと考えられるため、オンライン学習を検討することも可能であると考えられる。

## 参考文献

[1] ゼロから作るDeep Learning ―Pythonで学ぶディープラーニングの理論と実装、オライリージャパン 斎藤 康毅, 2016
[2] Mathworks, "畳み込みニューラル ネットワークとは", https://jp.mathworks.com/discovery/convolutional-neural-network-matlab.html
[3] Mathworks, "リカレントニューラルネットワーク (RNN) とは", https://jp.mathworks.com/discovery/rnn.html
[4] Mathwords, "バギングの意味", https://mathwords.net/bagging
